In [2]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
from sklearn.model_selection import train_test_split

In [1]:
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

tf.random.set_seed(42)

# import optuna

# import xgboost as xgb
# xgb.set_config(verbosity=2)

# from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# print(tf.config.list_physical_devices('GPU'))

KeyboardInterrupt: 

# Train data `all_more`

In [7]:
df_train = pd.read_csv("data/train_reviews_merged_all_more_prueba.csv")
df_train.sample(5)

,stars,useful,funny,cool,text,stars_neg,review_count_neg,is_open_neg,distancia_ref_neg,state_AZ_neg,...,funny_user,cool_user,fans_user,average_stars_user,antiguedad_anios_user,reviews_por_anio_user,num_friends_user,influencia_social_user,num_elite_years_user,compliment_user
323838,5.0,-0.391093,-0.150312,-0.251644,This place was amazing. I really enjoyed most ...,0.930844,8.171478,1,-1.341601,0,...,-0.046773,-0.031748,0.013048,1.231480,-0.045353,0.222573,-0.276150,-0.241869,0.453403,-0.033875
381566,5.0,-0.391093,-0.150312,-0.251644,"Fantastic food, made from simple, whole ingred...",-0.091512,0.424781,1,1.844137,0,...,-0.049962,-0.053261,-0.092248,1.068873,0.362122,-0.298660,0.037829,0.017687,-0.241889,-0.035085
360441,3.0,-0.062341,-0.150312,0.255613,The drive-thru lines can be long during the we...,0.419666,0.305966,1,-1.071168,0,...,0.179704,0.354355,1.101109,0.341925,1.460960,2.334731,0.131472,0.372080,6.711033,0.225173
290576,1.0,0.595161,-0.150312,-0.251644,Regarding the article by staff writer Soe Han ...,-1.625046,-0.224738,1,1.980661,0,...,-0.043583,-0.046468,-0.022051,0.762789,0.988110,-0.114531,-0.237591,-0.221903,-0.241889,-0.033875
196846,4.0,-0.391093,-0.150312,-0.251644,A great place for parents to take the kids and...,0.419666,1.256481,1,-0.665988,0,...,-0.051557,-0.053261,-0.092248,0.781920,-0.028337,-0.325392,-0.342251,-0.326723,-0.241889,-0.035085


In [8]:
df_train.shape

(967784, 51)

In [9]:
df_train.isnull().sum()

stars                        0
useful                       0
funny                        0
cool                         0
text                         0
stars_neg                    0
review_count_neg             0
is_open_neg                  0
distancia_ref_neg            0
state_AZ_neg                 0
state_FL_neg                 0
state_IN_neg                 0
state_LA_neg                 0
state_MO_neg                 0
state_NJ_neg                 0
state_Other_neg              0
state_PA_neg                 0
state_TN_neg                 0
open_weekends_neg            0
open_late_neg                0
group_restaurant_neg         0
group_beauty_neg             0
group_automotive_neg         0
group_food_neg               0
group_hotels_travel_neg      0
group_local_services_neg     0
group_shopping_neg           0
group_health_neg             0
group_entertainment_neg      0
group_pet_neg                0
attr_payment_neg             0
attr_appointment_neg         0
attr_tak

Cogo lo que me interesa y no sea texto o target

In [10]:
X_numeric = df_train.copy()

X_numeric = X_numeric.drop(columns=["stars", "text"])

X_numeric.sample(5)

,useful,funny,cool,stars_neg,review_count_neg,is_open_neg,distancia_ref_neg,state_AZ_neg,state_FL_neg,state_IN_neg,...,funny_user,cool_user,fans_user,average_stars_user,antiguedad_anios_user,reviews_por_anio_user,num_friends_user,influencia_social_user,num_elite_years_user,compliment_user
30315,1.252664,0.751859,2.284639,0.419666,4.511995,0,0.048378,0,0,0,...,3.536977,3.960629,3.452726,0.093232,1.563052,5.196172,2.153057,2.528388,6.015741,1.134262
770450,-0.391093,-0.150312,-0.251644,0.930844,44.964332,1,0.049541,0,0,0,...,-0.051557,-0.053261,-0.092248,-0.652846,0.657653,-0.302042,-0.336743,-0.321732,-0.241889,-0.035085
860109,-0.391093,-0.150312,-0.251644,-0.091512,2.373336,1,0.044122,0,0,0,...,0.256259,0.517402,1.136208,0.160188,-0.043562,3.947983,2.659831,2.618234,3.234572,0.125912
95770,-0.062341,-0.150312,-0.251644,0.930844,3.197116,1,-0.654216,0,0,0,...,-0.040393,-0.045335,-0.092248,0.408881,0.641533,-0.041430,-0.342251,-0.326723,-0.241889,-0.032664
222202,-0.391093,-0.150312,-0.251644,0.930844,4.092185,1,-1.338704,0,0,0,...,-0.019659,-0.041938,-0.092248,-0.882409,-0.044457,-0.099762,-0.342251,-0.326723,-0.241889,-0.032664


In [11]:
X_numeric.isnull().sum()

useful                       0
funny                        0
cool                         0
stars_neg                    0
review_count_neg             0
is_open_neg                  0
distancia_ref_neg            0
state_AZ_neg                 0
state_FL_neg                 0
state_IN_neg                 0
state_LA_neg                 0
state_MO_neg                 0
state_NJ_neg                 0
state_Other_neg              0
state_PA_neg                 0
state_TN_neg                 0
open_weekends_neg            0
open_late_neg                0
group_restaurant_neg         0
group_beauty_neg             0
group_automotive_neg         0
group_food_neg               0
group_hotels_travel_neg      0
group_local_services_neg     0
group_shopping_neg           0
group_health_neg             0
group_entertainment_neg      0
group_pet_neg                0
attr_payment_neg             0
attr_appointment_neg         0
attr_takeout_delivery_neg    0
attr_parking_neg             0
attr_wif

In [12]:
columns = ["text"]

df_train[columns].sample(5)

,text
704936,"Excellent service Very very professional, they..."
587922,I'm visiting from Idaho. Ashley was an amazin...
779622,I have been going to the Reno Academy on a reg...
572892,"Contrary to popular belief, snow cones are act..."
245474,"Had dinner here tonight. My advice, don't mak..."


## Procesando texto de manera clásica

In [ ]:
from scipy.sparse import hstack

# Aplicamos TfidfVectorizer en cada columna
tfidf_text = TfidfVectorizer(stop_words='english', max_features=15_000, ngram_range=(1, 2))
# tfidf_attr = TfidfVectorizer(stop_words='english', max_features=7_500, ngram_range=(1, 2))
# tfidf_cat  = TfidfVectorizer(stop_words='english', max_features=7_500, ngram_range=(1, 2))

X_text = tfidf_text.fit_transform(df_train['text'])
# X_attr = tfidf_attr.fit_transform(df_train['attributes_neg'])
# X_cat  = tfidf_cat.fit_transform(df_train['categories_neg'])

# Combinamos las matrices (cada una es una matriz dispersa)
# X_combined = hstack([X_text, X_attr, X_cat])

svd = TruncatedSVD(n_components=150, random_state=42)
X_text_reduced = svd.fit_transform(X_text)

In [5]:
X_text_reduced.shape, X_numeric.shape, X_text.shape

((967784, 150), (967784, 49), (967784, 15000))

In [5]:
X_final = np.hstack([X_numeric.values, X_text_reduced])

y = df_train["stars"]

X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((774227, 199), (193557, 199), (774227,), (193557,))

## Procesando texto DL

In [ ]:
# Combinamos las matrices (cada una es una matriz dispersa)
X_text = np.load("weight/embeddings_text_train_DL_all-mpnet-base-v2.npy")

# svd = TruncatedSVD(n_components=150, n_iter=20, n_oversamples=4, random_state=42)
# X_text_reduced = svd.fit_transform(X_text)

In [ ]:
X_text_reduced.shape, X_numeric.shape, X_text_reduced.shape

In [ ]:
X_final = np.load("weight/final_stack_prueba.npy")

y = df_train["stars"]

X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [12]:
np.save("final_stack.npy", X_final)

## RandomForestReg

In [ ]:
rf = RandomForestRegressor(n_estimators=50, random_state=42, verbose=1, n_jobs=-1)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

In [ ]:
importances = rf.feature_importances_

numeric_feature_names = list(X_numeric.columns)
text_feature_names = [f"text_comp_{i+1}" for i in range(X_text_reduced.shape[1])]
feature_names = numeric_feature_names + text_feature_names

indices = np.argsort(importances)[::-1]

# Crear la gráfica
plt.figure(figsize=(12, 8))
plt.title("Importancia de las características")
plt.bar(range(len(importances)), importances[indices], align="center")
plt.xticks(range(len(importances)), np.array(feature_names)[indices], rotation=90)
plt.xlabel("Características")
plt.ylabel("Importancia")
plt.tight_layout()
plt.show()

## KNeighborsRegressor

In [11]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_jobs=-1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

MAE: 0.5949162262279329


## HistGradientBoostingRegressor

In [10]:
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor(random_state=42, learning_rate=0.01, verbose=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

Binning 0.334 GB of training data: 0.458 s
Binning 0.037 GB of validation data: 0.030 s
Fitting gradient boosted rounds:
Fit 100 trees in 7.761 s, (3100 total leaves)
Time spent computing histograms: 5.310s
Time spent finding best splits:  0.234s
Time spent applying splits:      0.532s
Time spent predicting:           0.146s
MAE: 0.8606863429656669


## GradientBoostingRegressor

In [25]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(random_state=42, learning_rate=0.01, verbose=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

      Iter       Train Loss   Remaining Time 
         1           2.1710           38.69m
         2           2.1531           38.11m
         3           2.1355           37.62m
         4           2.1182           37.27m
         5           2.1013           36.81m
         6           2.0849           36.36m
         7           2.0685           35.97m
         8           2.0528           35.58m
         9           2.0370           35.20m
        10           2.0217           34.82m
        20           1.8810           32.45m
        30           1.7604           28.58m
        40           1.6572           24.20m
        50           1.5675           20.18m
        60           1.4896           16.26m
        70           1.4211           12.14m
        80           1.3607            8.05m
        90           1.3073            4.01m
       100           1.2604            0.00s
MAE: 0.938238277517477


## AdaBoostRegressor

In [27]:
from sklearn.ensemble import AdaBoostRegressor

ada = AdaBoostRegressor(random_state=42, learning_rate=0.001, n_estimators=4, 
                          estimator=RandomForestRegressor(n_estimators=50, random_state=42, verbose=1, n_jobs=-1))

ada.fit(X_train, y_train)

y_pred = ada.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  6.0min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  5.7min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  2.9

MAE: 0.5645549320941221


[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.4s finished


## XGBRegressor

In [16]:
xgboost = XGBRegressor(
    tree_method="hist",
    eval_metric=mean_absolute_error,
)

xgboost.fit(X_train, y_train, verbose=1)
y_pred = xgboost.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

[17:06:36] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 149, 115359823).
MAE: 0.5207063399151409


In [19]:
!pip install tensorflow_decision_forests==1.8.0

  Using cached protobuf-4.25.6-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.6 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5.1:
      Successfully uninstalled ml_dtypes-0.5.1
  Attempting uninstall: keras
    Found existing installation: keras 3.9.2
    Uninstalling keras-3.9.2:
      Successfully uninstalled keras-3.9.2
  Att

## Ensemble

In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Average, Lambda, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import gelu

In [ ]:
# --- 0. Datos y preprocesamiento común ---
num_features = X_train.shape[1]
raw_inputs = Input(shape=(num_features,), name="raw_inputs")

# Preprocesamiento simple: 184→32 reducidas a 10 dims
preprocessor = Dense(10, activation="relu6", name="preprocessor")
preprocessed = preprocessor(raw_inputs)
preprocessed

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'preprocessor')>

In [ ]:
# --- 1. Ramas NN idénticas (sin cambios) ---
def build_nn_branch(name):
    x = Dense(184, activation="relu", name=f"{name}_d1")(preprocessed)
    x = Dropout(0.3787, name=f"{name}_do1")(x)
    x = Dense(78, activation="relu", name=f"{name}_d2")(x)
    x = Dropout(0.1474, name=f"{name}_do2")(x)
    x = Dense(32, activation="relu", name=f"{name}_d3")(x)
    out = Dense(1, activation="linear", name=f"{name}_out")(x)
    return Model(raw_inputs, out, name=name)

nn1 = build_nn_branch("nn1")
nn2 = build_nn_branch("nn2")

for nn in (nn1, nn2):
    nn.compile(optimizer=Adam(3.403e-4), loss="mean_squared_error", metrics=["mae"])
    nn.fit(X_train, y_train,
           validation_data=(X_test, y_test),
           epochs=15, batch_size=16, verbose=1)

Epoch 1/15


In [ ]:
# --- 2. Ramas RandomForest y Transformer ---

# 2.1 RandomForest único (rf1)
def seed_args(seed):
    return tfdf.keras.AdvancedArguments(
        yggdrasil_training_config=tfdf.keras.core.YggdrasilTrainingConfig(random_seed=seed)
    )

rf1 = tfdf.keras.RandomForestModel(
    num_threads=15,
    num_trees=1000,
    advanced_arguments=seed_args(1234),
    name="rf1"
)

# Genera X_pp con la capa preprocessor
preproc_fn = tf.keras.backend.function([raw_inputs], [preprocessed])
X_train_pp = preproc_fn([X_train])[0]
X_val_pp   = preproc_fn([X_test])[0]

rf1.fit(x=X_train_pp, y=y_train, verbose=1)
m3_pred = rf1(preprocessed)

Use /tmp/tmp2vtj9ba7 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:17.970208. Found 774227 examples.
Training model...


[INFO 25-04-18 09:46:49.5596 UTC kernel.cc:1233] Loading model from path /tmp/tmp2vtj9ba7/model/ with prefix 4ee75f2847c14dc6
[INFO 25-04-18 09:47:40.9975 UTC decision_forest.cc:660] Model loaded with 1000 root(s), 29797968 node(s), and 10 input feature(s).
[INFO 25-04-18 09:47:40.9979 UTC abstract_model.cc:1344] Engine "RandomForestGeneric" built
[INFO 25-04-18 09:47:40.9979 UTC kernel.cc:1061] Use fast generic engine


Model trained in 0:06:59.924397
Compiling model...
Model compiled.


In [ ]:
rf1.save("weight/rf1_saved_model")

INFO:tensorflow:Assets written to: weight/rf1_saved_model/assets


INFO:tensorflow:Assets written to: weight/rf1_saved_model/assets


In [ ]:
rf1 = tf.keras.models.load_model("weight/rf1_saved_model")
rf1

[INFO 25-04-18 10:38:02.5071 UTC kernel.cc:1233] Loading model from path weight/rf1_saved_model/assets/ with prefix 4ee75f2847c14dc6
[INFO 25-04-18 10:38:43.1248 UTC decision_forest.cc:660] Model loaded with 1000 root(s), 29797968 node(s), and 10 input feature(s).
[INFO 25-04-18 10:38:43.1249 UTC abstract_model.cc:1344] Engine "RandomForestGeneric" built
[INFO 25-04-18 10:38:43.1249 UTC kernel.cc:1061] Use fast generic engine


In [ ]:
# 2.2 Transformer branch (sustituyendo rf2)
# Reusa raw_inputs para integrarlo en el mismo grafo
x = Dense(256, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(raw_inputs)
x = LayerNormalization()(x)
x = Lambda(lambda t: tf.expand_dims(t, axis=1))(x)

# Primera capa de MultiHeadAttention + residual
attn = MultiHeadAttention(num_heads=8, key_dim=128)(x, x)
attn = Dropout(0.4)(attn)
# residual y normalización
attn = Add()([tf.squeeze(x, axis=1), tf.squeeze(attn, axis=1)])
attn = LayerNormalization()(attn)

# Feed‑forward residual
ff = Dense(256, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attn)
ff = Dropout(0.4)(ff)
attn = Add()([attn, ff])
attn = LayerNormalization()(attn)

# Capas FFN profundas
ffn = Dense(256, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attn)
ffn = Dropout(0.4)(ffn)
ffn = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn)
ffn = Dropout(0.4)(ffn)
ffn = Dense(64, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn)
ffn = Dropout(0.4)(ffn)
ffn = Dense(32, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn)

# Salida
transformer_out = Dense(1, activation="linear", name="transformer_out")(ffn)
transformer = Model(raw_inputs, transformer_out, name="transformer")

# Compilación y entrenamiento
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-4,
    decay_steps=10_000,
    decay_rate=0.9
)
transformer.compile(optimizer=Adam(learning_rate=lr_schedule),
                    loss="mean_squared_error",
                    metrics=["mae"])
transformer.fit(X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=5, batch_size=16, verbose=1)

Epoch 1/5


I0000 00:00:1744970627.233226    6346 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


48390/48390 [==============================] - 408s 8ms/step - loss: 0.8382 - mae: 0.5172 - val_loss: 0.4959 - val_mae: 0.4237
Epoch 2/5
48390/48390 [==============================] - 393s 8ms/step - loss: 0.4906 - mae: 0.4651 - val_loss: 0.4731 - val_mae: 0.4079
Epoch 3/5
48390/48390 [==============================] - 415s 9ms/step - loss: 0.4550 - mae: 0.4495 - val_loss: 0.4353 - val_mae: 0.3991
Epoch 4/5
48390/48390 [==============================] - 416s 9ms/step - loss: 0.4306 - mae: 0.4382 - val_loss: 0.4141 - val_mae: 0.3904
Epoch 5/5
48390/48390 [==============================] - 401s 8ms/step - loss: 0.4139 - mae: 0.4302 - val_loss: 0.4063 - val_mae: 0.3745


In [ ]:
transformer.save("weight/transformer_saved_model")

INFO:tensorflow:Assets written to: weight/transformer_saved_model/assets


INFO:tensorflow:Assets written to: weight/transformer_saved_model/assets


In [ ]:
transformer = tf.keras.models.load_model("weight/transformer_saved_model")
transformer

In [ ]:
# --- 3. Ensamblaje final con 4 ramas: nn1, nn2, rf1 y transformer ---
avg_all = Average(name="avg_all")([
    nn1(raw_inputs),
    nn2(raw_inputs),
    m3_pred,
    transformer(raw_inputs)
])

ensemble_full = Model(raw_inputs, avg_all, name="ensemble_full")
ensemble_full.compile(
    optimizer=Adam(3.403e-4),
    loss="mean_squared_error",
    metrics=["mae"]
)
ensemble_full.summary()

In [ ]:
# --- 4. Evaluación ---
print("Ensamble (NNs + RF1 + Transformer):")
ensemble_full.evaluate(X_test, y_test, batch_size=16)

In [ ]:
ensemble_full.save("weight/ensemble_full_saved_model")

## Stacking

In [10]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

estimators = [
    ('ada', AdaBoostRegressor(random_state=42, learning_rate=0.001, n_estimators=6, 
                          estimator=XGBRegressor(tree_method="hist", eval_metric=mean_absolute_error))),

    ('rf', RandomForestRegressor(n_estimators=50, random_state=42, verbose=1, n_jobs=-1)),

    ('xgb', XGBRegressor(tree_method="hist", eval_metric=mean_absolute_error)),
]

stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression(),
    verbose=1
)

stacking_model.fit(X_train, y_train)
y_pred = stacking_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

[15:08:16] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 159, 123102093).
[15:08:29] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 159, 123102093).
[15:08:40] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 159, 123102093).
[15:08:52] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 159, 123102093).
[15:09:04] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 159, 123102093).
[15:09:15] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 159, 123102093).


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 24.5min finished


[15:33:56] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 159, 123102093).
[15:34:08] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619381, 159, 98481579).
[15:34:22] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619381, 159, 98481579).
[15:34:33] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619381, 159, 98481579).
[15:34:48] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619381, 159, 98481579).
[15:34:59] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619381, 159, 98481579).
[15:35:15] INFO: C:\actions

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 17.6min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 17.6min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  9.0

[17:07:05] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619381, 159, 98481579).
[17:07:16] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619381, 159, 98481579).
[17:07:26] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619382, 159, 98481738).
[17:07:36] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619382, 159, 98481738).
[17:07:46] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (619382, 159, 98481738).


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s


MAE: 0.4957841627129702


[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.8s finished


In [14]:
import joblib

# Guardar el modelo entrenado
joblib.dump(stacking_model, 'stacking_model_0.4957.pkl')

['stacking_model_0.4957.pkl']

XGBoost

In [40]:
loaded_model = joblib.load('../weight/stacking_model_0.546.pkl')
loaded_model

StackingRegressor(estimators=[('ada',
                               AdaBoostRegressor(estimator=RandomForestRegressor(n_estimators=50,
                                                                                 n_jobs=-1,
                                                                                 random_state=42,
                                                                                 verbose=1),
                                                 learning_rate=0.001,
                                                 n_estimators=4,
                                                 random_state=42)),
                              ('rf',
                               RandomForestRegressor(n_estimators=50, n_jobs=-1,
                                                     random_state=42,
                                                     verbose=1)),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_b...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...))],
                  final_estimator=LinearRegression(), verbose=1)

## VotingRegressor

In [ ]:
from sklearn.ensemble import VotingRegressor

xgboost = XGBRegressor(
    tree_method="hist",
    eval_metric=mean_absolute_error,
)

ada = AdaBoostRegressor(random_state=42, learning_rate=0.001, n_estimators=4, 
                          estimator=RandomForestRegressor(n_estimators=50, random_state=42, verbose=1, n_jobs=-1))

rf = RandomForestRegressor(n_estimators=50, random_state=42, verbose=1, n_jobs=-1)

er = VotingRegressor([('ada', ada), ('rf', rf), ('xgboost', xgboost)], n_jobs=-1)

er.fit(X_train, y_train)

VotingRegressor(estimators=[('ada',
                             AdaBoostRegressor(estimator=RandomForestRegressor(n_estimators=50,
                                                                               n_jobs=-1,
                                                                               random_state=42,
                                                                               verbose=1),
                                               learning_rate=0.001,
                                               n_estimators=4,
                                               random_state=42)),
                            ('rf',
                             RandomForestRegressor(n_estimators=50, n_jobs=-1,
                                                   random_state=42,
                                                   verbose=1)),
                            ('xgboost',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample...
                                          feature_weights=None, gamma=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None, ...))],
                n_jobs=-1)

In [48]:
y_pred = er.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0

MAE: 0.5514425473709493


[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.5s finished


## NeuralNetwork (baseline)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

tf.random.set_seed(42)

# [I 2025-04-16 16:53:27,772] Trial 1 finished with value: 0.40228334069252014 and parameters: {'n_units1': 184, 'dropout1': 0.3787372836263776, 'n_units2': 78, 
# 'dropout2': 0.14736679626439325, 'learning_rate': 0.00034032526493336896, 'batch_size': 16}. Best is trial 1 with value: 0.40228334069252014.


# Definir la arquitectura del modelo
input_dim = X_train.shape[1]  # 199 características
model = Sequential()
model.add(Dense(184, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.3787372836263776))
model.add(Dense(78, activation='relu'))
model.add(Dropout(0.14736679626439325))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # Capa de salida para regresión

# Compilar el modelo con Adam y error cuadrático medio
model.compile(optimizer=Adam(learning_rate=0.00034032526493336896),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 184)               150512    
                                                                 
 dropout (Dropout)           (None, 184)               0         
                                                                 
 dense_1 (Dense)             (None, 78)                14430     
                                                                 
 dropout_1 (Dropout)         (None, 78)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2528      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 167503 (654.31 KB)
Trainable params: 1675

In [ ]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=10, 
                    batch_size=16)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/10


48390/48390 [==============================] - 109s 2ms/step - loss: 0.4629 - mae: 0.4860 - val_loss: 0.3445 - val_mae: 0.4024
Epoch 2/10
48390/48390 [==============================] - 105s 2ms/step - loss: 0.3598 - mae: 0.4203 - val_loss: 0.3350 - val_mae: 0.3914
Epoch 3/10
48390/48390 [==============================] - 107s 2ms/step - loss: 0.3462 - mae: 0.4106 - val_loss: 0.3249 - val_mae: 0.3869
Epoch 4/10
48390/48390 [==============================] - 106s 2ms/step - loss: 0.3379 - mae: 0.4054 - val_loss: 0.3178 - val_mae: 0.3878
Epoch 5/10
48390/48390 [==============================] - 106s 2ms/step - loss: 0.3320 - mae: 0.4006 - val_loss: 0.3153 - val_mae: 0.3849
Epoch 6/10
48390/48390 [==============================] - 106s 2ms/step - loss: 0.3273 - mae: 0.3969 - val_loss: 0.3182 - val_mae: 0.3876
Epoch 7/10
48390/48390 [==============================] - 106s 2ms/step - loss: 0.3225 - mae: 0.3936 - val_loss: 0.3097 - val_mae: 0.3778
Epoch 8/10
48390/48390 [========

## Neural Network (2)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, BatchNormalization, LeakyReLU, Add
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

input_dim = X_train.shape[1]

# --- Bloque residual denso  ---
def dense_res_block(x, units, l2_reg=1e-4, dropout_rate=0.2):
    shortcut = x
    x = Dense(units, kernel_regularizer=l2(l2_reg), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(dropout_rate)(x)
    # Si cambia la dimensión, proyectamos shortcut
    if shortcut.shape[-1] != units:
        shortcut = Dense(units, kernel_regularizer=l2(l2_reg), use_bias=False)(shortcut)
    x = Add()([x, shortcut])
    return x

# --- Definición del modelo ---
inp = Input(shape=(input_dim,))
x = BatchNormalization()(inp)

# Primer bloque denso
x = Dense(256, kernel_regularizer=l2(1e-4), use_bias=False)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.2)(x)

# Dos bloques residuales
x = dense_res_block(x, units=128, l2_reg=1e-4, dropout_rate=0.15)
x = dense_res_block(x, units=64,  l2_reg=1e-4, dropout_rate=0.15)

# Cabeza final
x = Dense(32, kernel_regularizer=l2(1e-4))(x)
x = LeakyReLU()(x)
outputs = Dense(1, activation='linear')(x)

model = Model(inp, outputs)
model.compile(
    optimizer=Adam(learning_rate=3e-4),
    loss='mean_squared_error',
    metrics=['mae']
)

model.summary()

In [ ]:
# --- Callbacks para entrenamiento ---
callbacks = [
    ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=5, min_lr=1e-6),
    EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)
]

# --- Entrenamiento ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=callbacks
)

## CNN

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

tf.random.set_seed(42)

# Número de características
input_dim = X_train.shape[1]  # 199

model = Sequential([
    # 1) reshape explícito: (batch, 199) → (batch, 199, 1)
    Reshape((input_dim, 1), input_shape=(input_dim,)),
    
    # 2) capas Conv1D
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    Dropout(0.3),
    Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
    Dropout(0.2),
    
    # 3) pooling global
    GlobalMaxPooling1D(),
    
    # 4) densas finales para regresión
    Dense(32, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='linear'),
])

model.compile(
    optimizer=Adam(learning_rate=0.00034),
    loss='mean_squared_error',
    metrics=['mae']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 817, 1)            0         
                                                                 
 conv1d (Conv1D)             (None, 817, 64)           256       
                                                                 
 dropout (Dropout)           (None, 817, 64)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 817, 32)           6176      
                                                                 
 dropout_1 (Dropout)         (None, 817, 32)           0         
                                                                 
 global_max_pooling1d (Glob  (None, 32)                0         
 alMaxPooling1D)                                                 
                                                        

In [ ]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=10, 
                    batch_size=16)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

## Merlin Models

In [24]:
!pip install merlin-models

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-cuda to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a

In [25]:
import pandas as pd
import merlin.models.tf as mm
from merlin.io.dataset import Dataset

# 1) Crear DataFrames a partir de tus arrays
feature_cols = [f"f_{i}" for i in range(X_train.shape[1])]
train_df = pd.DataFrame(X_train, columns=feature_cols)
train_df["target"] = y_train  # tu columna de clase: valores en {0,1,2,3,4} o {1–5}
valid_df = pd.DataFrame(X_test, columns=feature_cols)
valid_df["target"] = y_test

# 2) Envolverlos en Dataset de Merlin
train = Dataset(train_df)
valid = Dataset(valid_df)

# 3) Definir el modelo DLRM para clasificación de 5 clases
model = mm.DLRMModel(
    train.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.MultiClassClassificationTask(
        target_name="target"   # la columna que contiene tus etiquetas
    )
)

# 4) Compilar, entrenar y evaluar
model.compile(optimizer="adagrad", run_eagerly=False)

model.fit(
    train,
    validation_data=valid,
    batch_size=1024,
    epochs=20,
)

metrics = model.evaluate(valid, batch_size=1024, return_dict=True)
print(metrics)

ImportError: cannot import name 'to_dlpack' from 'tensorflow.python' (/usr/local/lib/python3.11/dist-packages/tensorflow/python/__init__.py)

## Transformer (baseline)

In [7]:
from tensorflow.keras.activations import gelu
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(128, activation=gelu)(inputs)

# Add a sequence length dimension for MultiHeadAttention
dense_proj_expanded = tf.expand_dims(dense_proj, axis=1)

# Add multi-head attention
attention_output = MultiHeadAttention(num_heads=2, key_dim=64)(dense_proj_expanded, dense_proj_expanded)
attention_output = Dropout(0.3)(attention_output)

# Remove the sequence length dimension after attention
attention_output = tf.squeeze(attention_output, axis=1)

# Add residual connection and layer normalization
attention_output = Add()([dense_proj, attention_output])
attention_output = LayerNormalization()(attention_output)

# Add a feed-forward network
ffn_output = Dense(64, activation=gelu)(attention_output)
ffn_output = Dropout(0.3)(ffn_output)
ffn_output = Dense(32, activation=gelu)(ffn_output)

# Output layer for regression
outputs = Dense(1, activation='linear')(ffn_output)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model with Adam optimizer and mean squared error loss
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 199)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          25600       ['input_1[0][0]']                
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 128)       0           ['dense[0][0]']                  
                                                                                                  
 multi_head_attention (MultiHea  (None, 1, 128)      66048       ['tf.expand_dims[0][0]',         
 dAttention)                                                      'tf.expand_dims[0][0]']     

In [8]:
tf.keras.backend.clear_session()
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=5, 
                    batch_size=32)

# Evaluar el modelo en el conjunto de test
tf.keras.backend.clear_session()
loss, mae = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 [==============================] - 137s 5ms/step - loss: 0.5696 - mae: 0.5328 - val_loss: 0.6855 - val_mae: 0.6671
Epoch 2/5
24195/24195 [==============================] - 136s 6ms/step - loss: 0.4939 - mae: 0.4856 - val_loss: 0.5557 - val_mae: 0.5939
Epoch 3/5
24195/24195 [==============================] - 138s 6ms/step - loss: 0.4777 - mae: 0.4752 - val_loss: 0.6000 - val_mae: 0.6167
Epoch 4/5
24195/24195 [==============================] - 137s 6ms/step - loss: 0.4681 - mae: 0.4694 - val_loss: 0.5552 - val_mae: 0.5866
Epoch 5/5
6049/6049 [==============================] - 16s 3ms/step - loss: 0.5631 - mae: 0.5948
Test Loss: 0.5631492137908936
Test MAE: 0.5948130488395691


In [7]:
from tensorflow.keras.activations import gelu
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(128, activation=gelu)(inputs)

# Add a sequence length dimension for MultiHeadAttention
dense_proj_expanded = tf.expand_dims(dense_proj, axis=1)

# Add multi-head attention
attention_output = MultiHeadAttention(num_heads=2, key_dim=64)(dense_proj_expanded, dense_proj_expanded)
attention_output = Dropout(0.3)(attention_output)

# Remove the sequence length dimension after attention
attention_output = tf.squeeze(attention_output, axis=1)

# Add residual connection and layer normalization
attention_output = Add()([dense_proj, attention_output])
attention_output = LayerNormalization()(attention_output)

# Add a feed-forward network
ffn_output = Dense(64, activation=gelu)(attention_output)
ffn_output = Dropout(0.3)(ffn_output)
ffn_output = Dense(32, activation=gelu)(ffn_output)

# Output layer for regression
outputs = Dense(1, activation='linear')(ffn_output)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile with a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.00001,
    decay_steps=10_000,
    decay_rate=0.9
)
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 199)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          25600       ['input_1[0][0]']                
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 128)       0           ['dense[0][0]']                  
                                                                                                  
 multi_head_attention (MultiHea  (None, 1, 128)      66048       ['tf.expand_dims[0][0]',         
 dAttention)                                                      'tf.expand_dims[0][0]']     

In [92]:
tf.keras.backend.clear_session()

In [8]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=5, 
                    batch_size=32)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 [==============================] - 151s 6ms/step - loss: 1.4134 - mae: 0.9141 - val_loss: 1.0696 - val_mae: 0.8681
Epoch 2/5
24195/24195 [==============================] - 151s 6ms/step - loss: 0.8220 - mae: 0.6943 - val_loss: 1.1538 - val_mae: 0.9090
Epoch 3/5
24195/24195 [==============================] - 147s 6ms/step - loss: 0.7149 - mae: 0.6390 - val_loss: 1.2018 - val_mae: 0.9358
Epoch 4/5
24195/24195 [==============================] - 147s 6ms/step - loss: 0.6684 - mae: 0.6129 - val_loss: 1.2520 - val_mae: 0.9589
Epoch 5/5
6049/6049 [==============================] - 14s 2ms/step - loss: 1.2611 - mae: 0.9629
Test Loss: 1.2610799074172974
Test MAE: 0.9628550410270691


## Transfomer + regularization

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.activations import gelu

# Adding a Transformer-based architecture with regularization to reduce overfitting
tf.keras.backend.clear_session()

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)

# Add a sequence length dimension for MultiHeadAttention
dense_proj_expanded = tf.expand_dims(dense_proj, axis=1)

# Add multi-head attention with dropout
attention_output = MultiHeadAttention(num_heads=4, key_dim=64)(dense_proj_expanded, dense_proj_expanded)
attention_output = Dropout(0.4)(attention_output)

# Remove the sequence length dimension after attention
attention_output = tf.squeeze(attention_output, axis=1)

# Add residual connection and layer normalization
attention_output = Add()([dense_proj, attention_output])
attention_output = LayerNormalization()(attention_output)

# Add a feed-forward network with regularization
ffn_output = Dense(64, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attention_output)
ffn_output = Dropout(0.4)(ffn_output)
ffn_output = Dense(32, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)

# Output layer for regression
outputs = Dense(1, activation='linear')(ffn_output)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compilar el modelo con Adam y error cuadrático medio
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 199)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          25600       ['input_1[0][0]']                
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 128)       0           ['dense[0][0]']                  
                                                                                                  
 multi_head_attention (MultiHea  (None, 1, 128)      131968      ['tf.expand_dims[0][0]',         
 dAttention)                                                      'tf.expand_dims[0][0]']     

In [10]:
tf.keras.backend.clear_session()
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=5, 
                    batch_size=32)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 [==============================] - 145s 6ms/step - loss: 0.8266 - mae: 0.5915 - val_loss: 0.6719 - val_mae: 0.5746
Epoch 2/5
24195/24195 [==============================] - 152s 6ms/step - loss: 0.6705 - mae: 0.5515 - val_loss: 0.6660 - val_mae: 0.5097
Epoch 3/5
24195/24195 [==============================] - 151s 6ms/step - loss: 0.6576 - mae: 0.5410 - val_loss: 0.6757 - val_mae: 0.5698
Epoch 4/5
24195/24195 [==============================] - 156s 6ms/step - loss: 0.6475 - mae: 0.5365 - val_loss: 0.6443 - val_mae: 0.5389
Epoch 5/5
6049/6049 [==============================] - 17s 3ms/step - loss: 0.6222 - mae: 0.5416
Test Loss: 0.6222161650657654
Test MAE: 0.5416368842124939


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.activations import gelu

# Adding a Transformer-based architecture with regularization to reduce overfitting

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)

# Add a sequence length dimension for MultiHeadAttention
dense_proj_expanded = tf.expand_dims(dense_proj, axis=1)

# Increase the number of attention heads and key dimensions
attention_output = MultiHeadAttention(num_heads=8, key_dim=128)(dense_proj_expanded, dense_proj_expanded)
attention_output = Dropout(0.3)(attention_output)

# Add another feed-forward layer
ffn_output = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attention_output)
ffn_output = Dropout(0.3)(ffn_output)
ffn_output = Dense(64, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)
ffn_output = Dropout(0.3)(ffn_output)
ffn_output = Dense(32, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)

# Compile with a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=10_000,
    decay_rate=0.9
)
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 199)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          25600       ['input_1[0][0]']                
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 128)       0           ['dense[0][0]']                  
                                                                                                  
 multi_head_attention (MultiHea  (None, 1, 128)      66048       ['tf.expand_dims[0][0]',         
 dAttention)                                                      'tf.expand_dims[0][0]']     

In [10]:
tf.keras.backend.clear_session()
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=5, 
                    batch_size=32)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 [==============================] - 139s 6ms/step - loss: 0.4349 - mae: 0.4485 - val_loss: 0.5664 - val_mae: 0.5999
Epoch 2/5
24195/24195 [==============================] - 134s 6ms/step - loss: 0.4304 - mae: 0.4450 - val_loss: 0.5556 - val_mae: 0.5931
Epoch 3/5
24195/24195 [==============================] - 138s 6ms/step - loss: 0.4284 - mae: 0.4435 - val_loss: 0.5624 - val_mae: 0.5982
Epoch 4/5
24195/24195 [==============================] - 137s 6ms/step - loss: 0.4270 - mae: 0.4426 - val_loss: 0.5599 - val_mae: 0.5968
Epoch 5/5
6049/6049 [==============================] - 14s 2ms/step - loss: 0.5614 - mae: 0.5976
Test Loss: 0.5614352226257324
Test MAE: 0.5976006388664246


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.activations import gelu
import tensorflow as tf
# Adding a Transformer-based architecture with regularization to reduce overfitting

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)

# Add a sequence length dimension for MultiHeadAttention
dense_proj_expanded = tf.expand_dims(dense_proj, axis=1)

# Increase the number of attention heads and key dimensions
attention_output = MultiHeadAttention(num_heads=8, key_dim=128)(dense_proj_expanded, dense_proj_expanded)
attention_output = Dropout(0.3)(attention_output)

# Add another feed-forward layer
ffn_output = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attention_output)
ffn_output = Dropout(0.3)(ffn_output)
ffn_output = Dense(64, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)
ffn_output = Dropout(0.3)(ffn_output)
ffn_output = Dense(32, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)


# Output layer for regression
outputs = Dense(1, activation='linear')(ffn_output)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile with a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.00001,
    decay_steps=10_000,
    decay_rate=0.9
)
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 199)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          25600       ['input_2[0][0]']                
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 128)       0           ['dense[0][0]']                  
                                                                                                  
 multi_head_attention (MultiHea  (None, 1, 128)      527488      ['tf.expand_dims[0][0]',         
 dAttention)                                                      'tf.expand_dims[0][0]']     

In [9]:
tf.keras.backend.clear_session()
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=5, 
                    batch_size=32)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 [==============================] - 192s 8ms/step - loss: 4.1024 - mae: 1.0146 - val_loss: 2.3403 - val_mae: 0.7784
Epoch 2/5
24195/24195 [==============================] - 183s 8ms/step - loss: 1.9737 - mae: 0.7322 - val_loss: 1.5796 - val_mae: 0.6801
Epoch 3/5
24195/24195 [==============================] - 186s 8ms/step - loss: 1.4835 - mae: 0.6572 - val_loss: 1.3079 - val_mae: 0.6516
Epoch 4/5
24195/24195 [==============================] - 185s 8ms/step - loss: 1.2816 - mae: 0.6272 - val_loss: 1.1824 - val_mae: 0.6433
Epoch 5/5
6049/6049 [==============================] - 27s 4ms/step - loss: 1.1126 - mae: 0.6382
Test Loss: 1.112573266029358
Test MAE: 0.6381887197494507


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.activations import gelu
import tensorflow as tf

# Adding a Transformer-based architecture with regularization to reduce overfitting

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)

# Add a sequence length dimension for MultiHeadAttention
dense_proj_expanded = tf.expand_dims(dense_proj, axis=1)

# Increase the number of attention heads and key dimensions
attention_output = MultiHeadAttention(num_heads=4, key_dim=64)(dense_proj_expanded, dense_proj_expanded)
attention_output = Dropout(0.3)(attention_output)

# Add another feed-forward layer
ffn_output = Dense(64, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attention_output)
ffn_output = Dropout(0.3)(ffn_output)
ffn_output = Dense(32, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)
ffn_output = Dropout(0.3)(ffn_output)
ffn_output = Dense(16, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)

# Output layer for regression
outputs = Dense(1, activation='linear')(ffn_output)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile with a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=10_000,
    decay_rate=0.9
)
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 199)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          25600       ['input_1[0][0]']                
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 128)       0           ['dense[0][0]']                  
                                                                                                  
 multi_head_attention (MultiHea  (None, 1, 128)      131968      ['tf.expand_dims[0][0]',         
 dAttention)                                                      'tf.expand_dims[0][0]']     

In [11]:
tf.keras.backend.clear_session()
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=5, 
                    batch_size=32)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 [==============================] - 192s 8ms/step - loss: 1.4597 - mae: 0.6956 - val_loss: 0.7645 - val_mae: 0.5805
Epoch 2/5
24195/24195 [==============================] - 188s 8ms/step - loss: 0.7176 - mae: 0.5529 - val_loss: 0.6536 - val_mae: 0.5384
Epoch 3/5
24195/24195 [==============================] - 188s 8ms/step - loss: 0.6584 - mae: 0.5360 - val_loss: 0.6446 - val_mae: 0.5365
Epoch 4/5
24195/24195 [==============================] - 186s 8ms/step - loss: 0.6323 - mae: 0.5276 - val_loss: 0.6071 - val_mae: 0.5204
Epoch 5/5
6049/6049 [==============================] - 26s 4ms/step - loss: 0.5896 - mae: 0.5050
Test Loss: 0.589614748954773
Test MAE: 0.5050389170646667


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import gelu
import tensorflow as tf

# Adding a Transformer-based architecture with reduced heads and additional attention layer

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.02))(inputs)
dense_proj = LayerNormalization()(dense_proj)  # Add layer normalization

# Add a sequence length dimension for MultiHeadAttention
dense_proj_expanded = tf.expand_dims(dense_proj, axis=1)

# First MultiHeadAttention layer with reduced heads
attention_output_1 = MultiHeadAttention(num_heads=4, key_dim=64)(dense_proj_expanded, dense_proj_expanded)
attention_output_1 = Dropout(0.4)(attention_output_1)

# Remove the sequence length dimension after attention
attention_output_1 = tf.squeeze(attention_output_1, axis=1)

# Add residual connection and layer normalization
attention_output_1 = Add()([dense_proj, attention_output_1])
attention_output_1 = LayerNormalization()(attention_output_1)

# Second MultiHeadAttention layer
attention_output_2 = tf.expand_dims(attention_output_1, axis=1)
attention_output_2 = MultiHeadAttention(num_heads=2, key_dim=64)(attention_output_2, attention_output_2)
attention_output_2 = Dropout(0.4)(attention_output_2)

# Remove the sequence length dimension after attention
attention_output_2 = tf.squeeze(attention_output_2, axis=1)

# Add residual connection and layer normalization
attention_output_2 = Add()([attention_output_1, attention_output_2])
attention_output_2 = LayerNormalization()(attention_output_2)

# Add a feed-forward network with increased regularization
ffn_output = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.02))(attention_output_2)
ffn_output = LayerNormalization()(ffn_output)  # Add layer normalization
ffn_output = Dropout(0.4)(ffn_output)
ffn_output = Dense(32, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.02))(ffn_output)

# Output layer for regression
outputs = Dense(1, activation='linear')(ffn_output)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile with a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,
    decay_steps=10_000,
    decay_rate=0.9
)
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 199)]        0           []                               
                                                                                                  
 dense_8 (Dense)                (None, 128)          25600       ['input_3[0][0]']                
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 128)         256         ['dense_8[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 tf.expand_dims_4 (TFOpLambda)  (None, 1, 128)       0           ['layer_normalization_8[0][

In [7]:
import tensorflow as tf

tf.keras.backend.clear_session()
# Move X_train and y_train to GPU
X_train_gpu = tf.convert_to_tensor(X_train)
y_train_gpu = tf.convert_to_tensor(y_train)
X_test_gpu = tf.convert_to_tensor(X_test)
y_test_gpu = tf.convert_to_tensor(y_test)

In [14]:
history = model.fit(X_train_gpu, y_train_gpu, 
                    validation_data=(X_test, y_test),
                    epochs=5, 
                    batch_size=32)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test_gpu, y_test_gpu, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 [==============================] - 295s 12ms/step - loss: 0.8559 - mae: 0.5746 - val_loss: 0.6462 - val_mae: 0.6017
Epoch 2/5
24195/24195 [==============================] - 294s 12ms/step - loss: 0.5945 - mae: 0.5228 - val_loss: 0.7473 - val_mae: 0.6842
Epoch 3/5
24195/24195 [==============================] - 292s 12ms/step - loss: 0.5685 - mae: 0.5083 - val_loss: 0.7389 - val_mae: 0.6843
Epoch 4/5
24195/24195 [==============================] - 294s 12ms/step - loss: 0.5511 - mae: 0.4997 - val_loss: 0.6352 - val_mae: 0.6276
Epoch 5/5
6049/6049 [==============================] - 39s 6ms/step - loss: 0.6664 - mae: 0.6512
Test Loss: 0.6663885116577148
Test MAE: 0.6511966586112976


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.activations import gelu

# Adding a Transformer-based architecture with regularization to reduce overfitting
tf.keras.backend.clear_session()

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)

# Add a sequence length dimension for MultiHeadAttention
dense_proj_expanded = tf.expand_dims(dense_proj, axis=1)

# Add multi-head attention with dropout
attention_output = MultiHeadAttention(num_heads=8, key_dim=128)(dense_proj_expanded, dense_proj_expanded)
attention_output = Dropout(0.4)(attention_output)

# Remove the sequence length dimension after attention
attention_output = tf.squeeze(attention_output, axis=1)

# Add residual connection and layer normalization
attention_output = Add()([dense_proj, attention_output])
attention_output = LayerNormalization()(attention_output)

# Add a feed-forward network with regularization
ffn_output = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attention_output)
ffn_output = Dropout(0.4)(ffn_output)
ffn_output = Dense(64, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)
ffn_output = Dropout(0.4)(ffn_output)
ffn_output = Dense(32, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)
ffn_output = Dropout(0.4)(ffn_output)
ffn_output = Dense(16, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)

# Output layer for regression
outputs = Dense(1, activation='linear')(ffn_output)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,
    decay_steps=10_000,
    decay_rate=0.9
)
# Compilar el modelo con Adam y error cuadrático medio
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 199)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          25600       ['input_1[0][0]']                
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 128)       0           ['dense[0][0]']                  
                                                                                                  
 multi_head_attention (MultiHea  (None, 1, 128)      527488      ['tf.expand_dims[0][0]',         
 dAttention)                                                      'tf.expand_dims[0][0]']     

In [16]:
history = model.fit(X_train_gpu, y_train_gpu, 
                    validation_data=(X_test, y_test),
                    epochs=5, 
                    batch_size=32)

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test_gpu, y_test_gpu, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 [==============================] - 189s 8ms/step - loss: 1.0712 - mae: 0.6150 - val_loss: 0.6572 - val_mae: 0.5111
Epoch 2/5
24195/24195 [==============================] - 207s 9ms/step - loss: 0.6676 - mae: 0.5377 - val_loss: 0.6273 - val_mae: 0.4997
Epoch 3/5
24195/24195 [==============================] - 201s 8ms/step - loss: 0.6311 - mae: 0.5249 - val_loss: 0.5901 - val_mae: 0.4936
Epoch 4/5
24195/24195 [==============================] - 217s 9ms/step - loss: 0.6102 - mae: 0.5186 - val_loss: 0.6033 - val_mae: 0.4826
Epoch 5/5
6049/6049 [==============================] - 21s 3ms/step - loss: 0.5736 - mae: 0.4678
Test Loss: 0.5736496448516846
Test MAE: 0.46783918142318726


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import gelu
import tensorflow as tf

# Clear previous session
tf.keras.backend.clear_session()

# Define the input layer
input_dim = X_train.shape[1]  # 199 features
inputs = Input(shape=(input_dim,))

# Add a dense layer to expand dimensions for attention
dense_proj = Dense(256, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
dense_proj = LayerNormalization()(dense_proj)

# Add a sequence length dimension for MultiHeadAttention
from tensorflow.keras.layers import Lambda
dense_proj_expanded = Lambda(lambda x: tf.expand_dims(x, axis=1))(dense_proj)

# Add multiple MultiHeadAttention layers with dropout
attention_output_1 = MultiHeadAttention(num_heads=8, key_dim=128)(dense_proj_expanded, dense_proj_expanded)
attention_output_1 = Dropout(0.4)(attention_output_1)
# attention_output_1 = tf.squeeze(attention_output_1, axis=1)
attention_output_1 = Add()([dense_proj, attention_output_1])
attention_output_1 = LayerNormalization()(attention_output_1)

attention_output_2 = Dense(256, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attention_output_1)
attention_output_2 = Dropout(0.4)(attention_output_2)
attention_output_2 = Add()([attention_output_1, attention_output_2])
attention_output_2 = LayerNormalization()(attention_output_2)

# Add a deeper feed-forward network with regularization
ffn_output = Dense(256, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(attention_output_2)
ffn_output = Dropout(0.4)(ffn_output)
ffn_output = Dense(128, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)
ffn_output = Dropout(0.4)(ffn_output)
ffn_output = Dense(64, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)
ffn_output = Dropout(0.4)(ffn_output)
ffn_output = Dense(32, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(0.01))(ffn_output)

# Output layer for regression
outputs = Dense(1, activation='linear')(ffn_output)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile with a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,
    decay_steps=10_000,
    decay_rate=0.9
)
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 199)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     51,200 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 256)       │        512 │ dense[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1, 256)    │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 256)    │  1,051,904 │ lambda[0][0],     │
│ (MultiHeadAttentio… │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 1, 256)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 1, 256)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 256)    │        512 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1, 256)    │     65,792 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 1, 256)    │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 1, 256)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 256)    │        512 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1, 256)    │     65,792 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 1, 256)    │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1, 128)    │     32,896 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 1, 128)    │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1, 64)     │      8,256 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 1, 64)     │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1, 32)     │      2,080 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1, 1)      │         33 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,279,489 (4.88 MB)

 Trainable params: 1,279,489 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import os

checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/training_1/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=5,
                    batch_size=32,
                    callbacks=[cp_callback])

# Evaluar el modelo en el conjunto de test
loss, mae = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/5
24195/24195 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.3703 - mae: 0.6537
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/training_1/cp.weights.h5
24195/24195 ━━━━━━━━━━━━━━━━━━━━ 155s 6ms/step - loss: 2.3702 - mae: 0.6537 - val_loss: 0.5644 - val_mae: 0.5416
Epoch 2/5
24186/24195 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5562 - mae: 0.5009
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/training_1/cp.weights.h5
24195/24195 ━━━━━━━━━━━━━━━━━━━━ 137s 6ms/step - loss: 0.5562 - mae: 0.5009 - val_loss: 0.5178 - val_mae: 0.4793
Epoch 3/5
24187/24195 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5246 - mae: 0.4865
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/training_1/cp.weights.h5
24195/24195 ━━━━━━━━━━━━━━━━━━━━ 143s 6ms/step - loss: 0.5246 - mae: 0.4865 - val_loss: 0.4941 - val_mae: 0.4855
Epoch 4/5
24185/24195 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5090 - mae: 0.4804
Epoch 4: saving model to /content/drive/MyDrive/Colab Note

## Pruebas ensemble casero

In [ ]:
# Cargar los dos CSV
df_x = pd.read_csv(r'D:\github\RECSYS\Practica 2\out\Ensemble_NN_Trans_predictions_all_0_DL.csv')
df_y = pd.read_csv(r'D:\github\RECSYS\Practica 2\out\NN_predictions_all_0_DL_Sin_Reduc.csv')

# Asegurarse de que ambos DataFrames estén alineados por 'review_id'
df_x = df_x.set_index('review_id')
df_y = df_y.set_index('review_id')

# Verificar que ambos tengan las mismas filas
if not df_x.index.equals(df_y.index):
    raise ValueError("Los archivos CSV no tienen los mismos 'review_id'.")

# Generar un nuevo DataFrame con la ponderación de las columnas 'stars'
weight_x = 0.6  # Peso para el primer CSV
weight_y = 0.4  # Peso para el segundo CSV

df_combined = pd.DataFrame({
    'review_id': df_x.index,
    'stars': (df_x['stars'] * weight_x + df_y['stars'] * weight_y).round(0).astype(float)
})

# Guardar el nuevo DataFrame en un archivo CSV
df_combined.to_csv('combined_stars.csv', index=False)
print("Archivo combinado exportado como 'combined_stars.csv'")

Archivo combinado exportado como 'combined_stars.csv'


In [ ]:
import numpy as np

# 1) Entrena cada modelo individual
models = [
    XGBRegressor(tree_method="hist", eval_metric='mae', random_state=42),
    AdaBoostRegressor(random_state=42, learning_rate=0.001, n_estimators=4,
               estimator=RandomForestRegressor(n_estimators=50, random_state=42, verbose=1, n_jobs=-1)),
    # tu wrapper de Keras u otro modelo,
    lgbm_reg = LGBMRegressor(
    objective='regression',
    boosting_type='gbdt',
    learning_rate=0.1,
    n_estimators=1000,
    num_leaves=31,
    random_state=42
    )
]
for m in models:
    m.fit(X_train, y_train)

# 2) Predice con cada uno
preds = [m.predict(X_test) for m in models]

# 3) Media aritmética simple
y_pred_avg = np.mean(preds, axis=0)

# 4) Evalúa
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred_avg)
print(f"MAE (average ensemble): {mae:.4f}")

In [ ]:
weights = [0.4, 0.1, 0.3, 0.2]  # suma 1.0
y_pred_w = np.average(preds, axis=0, weights=weights)
print("MAE (weighted):", mean_absolute_error(y_test, y_pred_w))

# Test models

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

KeyboardInterrupt: 

In [11]:
trans = tf.keras.models.load_model(filepath=r'D:\github\RECSYS\Practica 2\weight\transformer_saved_model')
trans

In [12]:
df_test = pd.read_csv(r'D:\github\RECSYS\Practica 2\data\test_reviews_merged_all_more_prueba.csv')
df_test.sample(5)

,review_id,useful,funny,cool,text,stars_neg,review_count_neg,is_open_neg,distancia_ref_neg,state_AZ_neg,...,funny_user,cool_user,fans_user,average_stars_user,antiguedad_anios_user,reviews_por_anio_user,num_friends_user,influencia_social_user,num_elite_years_user,compliment_user
403628,EhGt3u8D9K1F90PjRD31HQ,0.266410,-0.150312,-0.251644,Very happy with the service here at Keith's! H...,0.930844,-0.264343,1,0.030301,0,...,0.214792,0.100727,0.434233,0.255839,1.569321,1.307456,0.136980,0.182405,-0.241889,0.078702
322664,cmz4KDov4tJ3SaMbJ0inoQ,-0.391093,-0.150312,-0.251644,"Wow, nuggets overpriced. Next time I will go t...",0.419666,0.290124,1,-0.129773,0,...,-0.024444,-0.041938,-0.057150,-1.217187,-0.227149,0.129114,-0.303692,-0.286792,-0.241889,-0.033875
18123,8_u9j4ral9-NF7zMr1pvrw,-0.062341,-0.150312,-0.251644,"When I worked in Cool Springs, this was my go-...",0.419666,0.020812,1,-1.337030,0,...,1.005864,0.724607,1.662689,0.045407,2.055604,3.930038,0.831039,1.065892,5.320449,0.380118
318998,Cef-h2Yk4sqKCE0QBmc2fw,0.266410,-0.150312,-0.251644,I highly recommend this place!! Lenny is great...,1.442022,0.361413,1,1.979742,0,...,-0.051557,-0.049864,-0.057150,-0.126765,-0.531636,-0.222025,-0.342251,-0.321732,-0.241889,-0.035085
411189,RJs_PeRi2LEeNa4RqTCxmg,-0.391093,-0.150312,-0.251644,I so want this place to do well. It's got a go...,0.419666,0.369334,0,0.049385,0,...,-0.051557,-0.053261,-0.092248,-0.461544,1.392898,-0.308700,-0.342251,-0.326723,-0.241889,-0.035085


In [13]:
Y_numeric = df_test.copy()

Y_numeric = Y_numeric.drop(columns=["text", "review_id"])

Y_numeric.sample(5)

,useful,funny,cool,stars_neg,review_count_neg,is_open_neg,distancia_ref_neg,state_AZ_neg,state_FL_neg,state_IN_neg,...,funny_user,cool_user,fans_user,average_stars_user,antiguedad_anios_user,reviews_por_anio_user,num_friends_user,influencia_social_user,num_elite_years_user,compliment_user
236977,-0.391093,0.300774,-0.251644,0.419666,0.955485,1,0.021202,0,0,0,...,0.013834,0.014675,0.188542,0.121927,1.961571,0.916518,0.026812,0.067601,1.148696,-0.014507
218810,-0.391093,-0.150312,-0.251644,0.419666,3.783267,1,-1.352899,0,0,0,...,-0.040393,-0.048732,-0.022051,-1.073711,1.437676,-0.241125,-0.044797,-0.047202,-0.241889,-0.031454
189405,-0.062341,-0.150312,0.255613,0.930844,1.549557,1,-1.082434,0,0,1,...,-0.048367,-0.038542,-0.022051,0.437576,0.950497,-0.187862,-0.226575,-0.211920,-0.241889,-0.033875
180711,-0.391093,-0.150312,-0.251644,0.930844,2.294127,1,-1.069237,0,0,1,...,-0.051557,-0.053261,-0.092248,1.260175,-0.190432,-0.335567,1.260695,1.125790,-0.241889,-0.035085
216614,-0.391093,-0.150312,-0.251644,0.419666,1.502031,0,0.049087,0,0,0,...,0.085604,0.005617,0.188542,-0.432849,1.539768,0.865981,-0.309201,-0.256843,-0.241889,0.003651


In [11]:
Y_text = np.load(r'D:\github\RECSYS\Practica 2\weight\embeddings_text_test_DL_all-mpnet-base-v2.npy')

# svd = TruncatedSVD(n_components=150, n_iter=20, n_oversamples=4, random_state=42)
# Y_text_test_svd = svd.fit_transform(Y_text)

X_test = np.hstack([Y_numeric.values, Y_text])

# np.save("X_test.npy", X_test)
predictions = model.predict(X_test)

NameError: name 'Y_numeric' is not defined

## CSV

In [ ]:
# Crear el DataFrame con el formato requerido: review_id, stars
df_output = pd.DataFrame({
    'review_id': df_test['review_id'],
    'stars': predictions.round(0)
})

# Exportar el CSV con las predicciones
df_output.to_csv('stacking_model_model_predictions_all_0_DL.csv', index=False)
print("Predicciones exportadas a 'xgboost_model_predictions_all_0_DL.csv'")

In [ ]:
# Crear el DataFrame con el formato requerido: review_id, stars
df_output = pd.DataFrame({
    'review_id': df_test['review_id'],
    'stars': predictions.squeeze().round(4)  # Asegúrate de que sea un número entero
})

# Exportar el CSV con las predicciones
df_output.to_csv('NN_predictions_all_1_DL_Sin_Reduc_without_round.csv', index=False)
print("Predicciones exportadas a 'NN_predictions_all_1_DL_Sin_Reduc_without_round.csv'")

Predicciones exportadas a 'Trans_predictions_all_1_DL_Sin_Reduc.csv'


# Hiperparámetros

## Reduccion dimensional (TruncatedSVD)

In [ ]:
def objective(trial):
    # Sugerencias para TruncatedSVD
    n_components = trial.suggest_int('n_components', 20, 200, step=5)
    n_iter = trial.suggest_int('n_iter', 5, 20, step=5)
    n_oversamples = trial.suggest_int('n_oversamples', 2, 8, step = 1)

    print(f"Trial {trial.number} - n_components: {n_components}, n_iter: {n_iter}, n_oversamples: {n_oversamples}")

    # Cargamos los embeddings precomputados
    X_text = np.load("../weight/embeddings_text_train_DL_all-mpnet-base-v2.npy")

    # Creamos el TruncatedSVD con los hiperparámetros sugeridos
    svd = TruncatedSVD(
        n_components=n_components,
        n_iter=n_iter,
        n_oversamples=n_oversamples,
        random_state=42
    )
    X_text_reduced = svd.fit_transform(X_text)
    
    # Combinamos con otras características numéricas
    X_final = np.hstack([X_numeric.values, X_text_reduced])
    
    y = df_train["stars"]
    X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

    # Entrenamos el modelo XGBoost
    reg = XGBRegressor(
        tree_method="hist",
        eval_metric=mean_absolute_error,
        random_state=42,
    )
    reg.fit(X_train, y_train, verbose=False)
    y_pred = reg.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"Trial {trial.number} - MAE: {mae}\n")
    return mae

In [11]:
# Creamos y ejecutamos el estudio de Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print("Mejores hiperparámetros encontrados:", study.best_trial.params)
print("Mejor MAE:", study.best_value)

[I 2025-04-13 13:12:27,242] A new study created in memory with name: no-name-e863853c-c69d-4051-906a-1e949f57f575


Trial 0 - n_components: 125, n_iter: 10, n_oversamples: 8
[13:13:20] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 134, 103746418).
Trial 0 - MAE: 0.5198457555800627



[I 2025-04-13 13:13:28,864] Trial 0 finished with value: 0.5198457555800627 and parameters: {'n_components': 125, 'n_iter': 10, 'n_oversamples': 8}. Best is trial 0 with value: 0.5198457555800627.


Trial 1 - n_components: 115, n_iter: 15, n_oversamples: 5
[13:14:29] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 124, 96004148).


[I 2025-04-13 13:14:35,923] Trial 1 finished with value: 0.5209107322448631 and parameters: {'n_components': 115, 'n_iter': 15, 'n_oversamples': 5}. Best is trial 0 with value: 0.5198457555800627.


...

Trial 28 - n_components: 140, n_iter: 20, n_oversamples: 3
[13:52:08] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 149, 115359823).
Trial 28 - MAE: 0.5195847145778667



[I 2025-04-13 13:52:16,493] Trial 28 finished with value: 0.5195847145778667 and parameters: {'n_components': 140, 'n_iter': 20, 'n_oversamples': 3}. Best is trial 5 with value: 0.5184367227010693.


Trial 29 - n_components: 130, n_iter: 20, n_oversamples: 3
[13:53:48] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (774227, 139, 107617553).
Trial 29 - MAE: 0.5192138864666965



[I 2025-04-13 13:53:56,539] Trial 29 finished with value: 0.5192138864666965 and parameters: {'n_components': 130, 'n_iter': 20, 'n_oversamples': 3}. Best is trial 5 with value: 0.5184367227010693.


Mejores hiperparámetros encontrados: {'n_components': 150, 'n_iter': 20, 'n_oversamples': 4}
Mejor MAE: 0.5184367227010693


## Reduccion dimensional (PCA)

In [ ]:
def objective(trial):
    n_components = trial.suggest_int('n_components', 20, 200, step=5)

    print(f"Trial {trial.number} - max_componets: {n_components}")

    X_text = np.load("../weight/embeddings_text_train_DL_all-mpnet-base-v2.npy")

    svd = PCA(n_components=n_components, random_state=42)
    X_text_reduced = svd.fit_transform(X_text)  
    
    X_final = np.hstack([X_numeric.values, X_text_reduced])
    
    y = df_train["stars"]
    
    X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

    # Entrenamos el modelo XGBoost
    reg = XGBRegressor(
        tree_method="hist",
        eval_metric=mean_absolute_error,
        random_state=42,
    )
    reg.fit(X_train, y_train, verbose=False)
    y_pred = reg.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    # Imprimimos el MAE obtenido para esta prueba
    print(f"Trial {trial.number} - MAE: {mae}\n")
    return mae

## XGBoost

In [35]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna
from xgboost import XGBRegressor

import xgboost as xgb
xgb.set_config(verbosity=0)

def objective(trial):
    # Cargamos las características y la variable objetivo
    X_final = np.load("../weight/final_stack_XGBoost.npy")
    y = df_train["stars"]
    X_train, X_test, y_train, y_test = train_test_split(
        X_final, y, test_size=0.2, random_state=42
    )
    
    # Mapeo para parámetros categóricos
    booster_options = {1: "gbtree", 2: "gblinear", 3: "dart"}
    tree_method_options = {1: "auto", 2: "hist"}
    grow_policy_options = {1: "depthwise", 2: "lossguide"}
    
    booster_idx = trial.suggest_int("booster_idx", 1, 3)
    tree_method_idx = trial.suggest_int("tree_method_idx", 1, 2)
    grow_policy_idx = trial.suggest_int("grow_policy_idx", 1, 2)
    
    booster = booster_options[booster_idx]
    tree_method = tree_method_options[tree_method_idx]
    grow_policy = grow_policy_options[grow_policy_idx]
    
    # Parámetros numéricos actualizados
    eta = trial.suggest_float("eta", 1e-3, 0.7, log=True)
    gamma = trial.suggest_float("gamma", 0.0, 10.0)
    max_depth = trial.suggest_int("max_depth", 3, 12)
    min_child_weight = trial.suggest_float("min_child_weight", 1, 5)
    max_delta_step = trial.suggest_float("max_delta_step", 0, 5)
    subsample = trial.suggest_float("subsample", 0.2, 1.0)
    reg_lambda = trial.suggest_float("lambda", 1e-3, 10.0, log=True)
    alpha = trial.suggest_float("alpha", 1e-3, 10.0, log=True)
    scale_pos_weight = trial.suggest_float("scale_pos_weight", 0.5, 5)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    max_bin = trial.suggest_int("max_bin", 125, 525, step=25)
    
    params = {
        "booster": booster,
        "tree_method": tree_method,
        "grow_policy": grow_policy,
        "eta": eta,
        "gamma": gamma,
        "max_depth": max_depth,
        "min_child_weight": min_child_weight,
        "max_delta_step": max_delta_step,
        "subsample": subsample,
        "reg_lambda": reg_lambda,
        "alpha": alpha,
        "scale_pos_weight": scale_pos_weight,
        "max_leaves": max_leaves,
        "max_bin": max_bin,
        "random_state": 42,
        "eval_metric": "mae",
        "device": "cuda",
    }
    
    print(f"Trial {trial.number} parameters: {params}")
    
    # Entrenamos el modelo
    reg = XGBRegressor(**params)
    reg.fit(X_train, y_train, verbose=False)
    y_pred = reg.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"Trial {trial.number} - MAE: {mae}\n")
    return mae

# Para iniciar el estudio:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)

[I 2025-04-13 17:30:59,386] A new study created in memory with name: no-name-c35bc967-1f54-4831-9a1d-eb5dd20ecce7


Trial 0 parameters: {'booster': 'gbtree', 'tree_method': 'hist', 'grow_policy': 'depthwise', 'eta': 0.05011492737137264, 'gamma': 6.7875345789727515, 'max_depth': 5, 'min_child_weight': 2.985288188657927, 'max_delta_step': 0.8120372013412991, 'subsample': 0.4019356632484379, 'reg_lambda': 0.00852567099520477, 'alpha': 2.0282623475373343, 'scale_pos_weight': 4.418004912122489, 'max_leaves': 3, 'max_bin': 500, 'random_state': 42, 'eval_metric': 'mae', 'device': 'cuda'}


[I 2025-04-13 17:31:06,144] Trial 0 finished with value: 0.7812142846968104 and parameters: {'booster_idx': 1, 'tree_method_idx': 2, 'grow_policy_idx': 1, 'eta': 0.05011492737137264, 'gamma': 6.7875345789727515, 'max_depth': 5, 'min_child_weight': 2.985288188657927, 'max_delta_step': 0.8120372013412991, 'subsample': 0.4019356632484379, 'lambda': 0.00852567099520477, 'alpha': 2.0282623475373343, 'scale_pos_weight': 4.418004912122489, 'max_leaves': 3, 'max_bin': 500}. Best is trial 0 with value: 0.7812142846968104.


Trial 0 - MAE: 0.7812142846968104

Trial 1 parameters: {'booster': 'gbtree', 'tree_method': 'auto', 'grow_policy': 'depthwise', 'eta': 0.1651613589789469, 'gamma': 4.879083371462409, 'max_depth': 3, 'min_child_weight': 2.3834085610483386, 'max_delta_step': 1.320883813843332, 'subsample': 0.9200176257999957, 'reg_lambda': 0.009227867220793355, 'alpha': 0.6149223137348641, 'scale_pos_weight': 4.58589918293612, 'max_leaves': 8, 'max_bin': 325, 'random_state': 42, 'eval_metric': 'mae', 'device': 'cuda'}


[I 2025-04-13 17:31:11,631] Trial 1 finished with value: 0.6192486519362881 and parameters: {'booster_idx': 1, 'tree_method_idx': 1, 'grow_policy_idx': 1, 'eta': 0.1651613589789469, 'gamma': 4.879083371462409, 'max_depth': 3, 'min_child_weight': 2.3834085610483386, 'max_delta_step': 1.320883813843332, 'subsample': 0.9200176257999957, 'lambda': 0.009227867220793355, 'alpha': 0.6149223137348641, 'scale_pos_weight': 4.58589918293612, 'max_leaves': 8, 'max_bin': 325}. Best is trial 1 with value: 0.6192486519362881.


...

Trial 301 - MAE: 0.552840997653414

Trial 302 parameters: {'booster': 'dart', 'tree_method': 'auto', 'grow_policy': 'lossguide', 'eta': 0.415169587097441, 'gamma': 0.3457530772631484, 'max_depth': 10, 'min_child_weight': 3.3500044082404066, 'max_delta_step': 2.677806069833423, 'subsample': 0.8904488310688177, 'reg_lambda': 1.2328599329040852, 'alpha': 0.0019085697805791939, 'scale_pos_weight': 0.7656232376076798, 'max_leaves': 10, 'max_bin': 200, 'random_state': 42, 'eval_metric': 'mae', 'device': 'cuda'}


[I 2025-04-13 18:21:37,751] Trial 302 finished with value: 0.5521674668585461 and parameters: {'booster_idx': 3, 'tree_method_idx': 1, 'grow_policy_idx': 2, 'eta': 0.415169587097441, 'gamma': 0.3457530772631484, 'max_depth': 10, 'min_child_weight': 3.3500044082404066, 'max_delta_step': 2.677806069833423, 'subsample': 0.8904488310688177, 'lambda': 1.2328599329040852, 'alpha': 0.0019085697805791939, 'scale_pos_weight': 0.7656232376076798, 'max_leaves': 10, 'max_bin': 200}. Best is trial 241 with value: 0.5493317376051099.


Trial 302 - MAE: 0.5521674668585461

Trial 303 parameters: {'booster': 'dart', 'tree_method': 'auto', 'grow_policy': 'lossguide', 'eta': 0.3785644103200653, 'gamma': 3.0038223908850608, 'max_depth': 9, 'min_child_weight': 3.106505709220485, 'max_delta_step': 2.358462438871258, 'subsample': 0.8580218411180177, 'reg_lambda': 1.057322169576392, 'alpha': 0.0013670160815387141, 'scale_pos_weight': 2.1632466388641944, 'max_leaves': 10, 'max_bin': 200, 'random_state': 42, 'eval_metric': 'mae', 'device': 'cuda'}


Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x00000248FC8DD490>>
Traceback (most recent call last):
  File "d:\github\RECSYS\.venv\Lib\site-packages\xgboost\core.py", line 585, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInterrupt: 
[W 2025-04-13 18:21:51,050] Trial 303 failed with parameters: {'booster_idx': 3, 'tree_method_idx': 1, 'grow_policy_idx': 2, 'eta': 0.3785644103200653, 'gamma': 3.0038223908850608, 'max_depth': 9, 'min_child_weight': 3.106505709220485, 'max_delta_step': 2.358462438871258, 'subsample': 0.8580218411180177, 'lambda': 1.057322169576392, 'alpha': 0.0013670160815387141, 'scale_pos_weight': 2.1632466388641944, 'max_leaves': 10, 'max_bin': 200} because of the following error: XGBoostError('[18:21:51] C:\\actions-runner\\_work\\xgboost\\xgboost\\src\\data\\quantile_dmatrix.cc:174: Check failed: accumulate

XGBoostError: [18:21:51] C:\actions-runner\_work\xgboost\xgboost\src\data\quantile_dmatrix.cc:174: Check failed: accumulated_rows == info.num_row_ (1548454 vs. 774227) : 

In [ ]:
print("Best trial:")
trial = study.best_trial
print(trial.params)
print(f"Best MAE: {trial.value}")

## Neural Network

In [ ]:
import optuna
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# [I 2025-04-16 16:53:27,772] Trial 1 finished with value: 0.40228334069252014 and parameters: {'n_units1': 184, 'dropout1': 0.3787372836263776, 'n_units2': 78, 
# 'dropout2': 0.14736679626439325, 'learning_rate': 0.00034032526493336896, 'batch_size': 16}. Best is trial 1 with value: 0.40228334069252014.

tf.random.set_seed(42)

# Suponiendo que X_train, y_train, X_test y y_test ya están definidos

def create_model(trial, input_dim):
    # Sugerimos algunos hiperparámetros:
    n_units1 = trial.suggest_int('n_units1', 64, 256)
    dropout1 = trial.suggest_float('dropout1', 0.1, 0.5)
    n_units2 = trial.suggest_int('n_units2', 32, 128)
    dropout2 = trial.suggest_float('dropout2', 0.1, 0.5)
    n_units3 = trial.suggest_int('n_units3', 16, 128)

    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    
    model = Sequential()
    model.add(Dense(n_units1, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout1))
    model.add(Dense(n_units2, activation='relu'))
    model.add(Dropout(dropout2))
    model.add(Dense(n_units3, activation='relu'))  
    model.add(Dense(1, activation='linear'))   
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['mae'])
    return model

def objective(trial):
    model = create_model(trial, input_dim=X_train.shape[1])
    
    # Utilizamos EarlyStopping basado en la métrica 'mae'
    early_stop = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)
    
    # Sugerimos un batch_size entre varias opciones
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    
    # Entrenamos el modelo con X_train, y_train y evaluamos en (X_test, y_test)
    model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,
                    batch_size=batch_size,
                    callbacks=[early_stop],
                    verbose=1)
    
    # Evaluamos el modelo en el conjunto de test para obtener el MAE
    loss, mae = model.evaluate(X_test, y_test, verbose=0)

    # Imprimir el MAE y los hiperparámetros usados en este trial
    print(f"Trial {trial.number} - MAE: {mae}")
    print("Hiperparámetros probados:", trial.params)
    
    return mae

# Creamos el estudio de Optuna para minimizar el MAE
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150)

[I 2025-04-17 13:15:32,831] A new study created in memory with name: no-name-d1f49861-b86c-41c9-804f-bd29218cb7ce


In [ ]:
print("Mejores hiperparámetros:", study.best_params)
print("Mejor MAE:", study.best_value)